In [ ]:
import requests
from bs4 import BeautifulSoup
from math import ceil, nan
import json, re, os

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    page = BeautifulSoup(req.text,"html.parser")
    return page

In [ ]:
url = ""
page = render_html(url)

In [ ]:
product_data = {
    "code" : "",
    "sources" : ["https://www.auchan.fr/auchan-saumon-fume-de-norvege-mini/pr-C1182526"],
    "image_url" : "https://media.auchan.fr/MEDIASTEP65568709_512x512/B2CD/",
    "small_image_url" : "https://media.auchan.fr/MEDIASTEP65568709_512x512/B2CD/",
    "product_name" : "",
    "brand" : "",
    "nutriscore" : "",
    "categories" : ["Accueil", "Boucherie"],
    "ingredients_text" : "",
    "calories" : "",
}